In [1]:
import pandas as pd

In [2]:
dfSensor = pd.read_pickle("./data/step01_sensor.pkl")

In [3]:
dfSensor["TimeReal"] = pd.to_datetime(dfSensor.Time,unit='ms')

In [47]:
%%time
lstdf = []
for pid in dfSensor.PId.unique():
    print(pid)
    for phone in dfSensor.Phone.unique():
        dfX = dfSensor[(dfSensor.PId == pid) & (dfSensor.Phone == phone)]
        dfX = dfX.set_index(["TimeReal", "Sensor"])
        dfX = dfX.unstack()
        dfX = dfX.groupby("TimeReal").mean()
        for x in dfSensor.Sensor.unique():
            del dfX[('Time', x)]
            del dfX[('PId', x)]
        dfX = dfX.fillna(method="pad")
        dfX = dfX.dropna()
        dfX = dfX.resample("3ms").pad()
        lst = [] 
        for x in ['X', 'Y', 'Z']:
            for y in ['acc', 'grav', 'gyro', 'mag', 'ori', 'rot']:
                lst.append(y.title()+x)
        dfX.columns= lst
        dfX["PId"] = pid
        dfX["Phone"] = phone
        dfX = dfX.reset_index()
        lstdf.append(dfX)

CPU times: user 7min 23s, sys: 2min 18s, total: 9min 41s
Wall time: 9min 41s


In [48]:
dfnew = pd.concat(lstdf)

In [51]:
dfnew.to_pickle("./data/step02_sensor.pkl")